# Day 10: Flower FL Framework

**Production-Ready Federated Learning Framework**

## Overview
- **Framework**: Flower (flwr) - friendly FL framework
- **Language**: Python, framework-agnostic
- **Use Case**: Production FL deployments

## What You'll Learn
1. **Flower Architecture**: Client-server FL with Flower
2. **Client Implementation**: Define federated learning client
3. **Server Configuration**: FedAvg with Flower server
4. **Strategy**: Custom aggregation strategies

---

## 1. Setup and Installation

In [ ]:
# Install Flower (if not already installed)
# !pip install flwr

import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (12, 6)

print("✅ Libraries imported!")
print("\nFlower Framework Overview:")
print("  • Language-agnostic: Python, Rust, Swift, Java, Go")
print("  • Framework-agnostic: Works with PyTorch, TensorFlow, JAX")
print("  • Production-ready: Used by major companies")

## 2. Flower Client Implementation

In [ ]:
# This demonstrates the Flower client API
# (Actual execution requires flwr package)

flower_client_code = '''
from flwr.client import NumPyClient, ClientApp
from flwr.common import Context
import numpy as np

class FraudDetectionClient(NumPyClient):
    """Flower client for fraud detection."""
    
    def __init__(self, model, X_train, y_train, X_test, y_test):
        self.model = model
        self.X_train = X_train
        self.y_train = y_train
        self.X_test = X_test
        self.y_test = y_test
    
    def get_parameters(self, config):
        """Return current model parameters."""
        # Return weights as list of numpy arrays
        return self.model.get_weights()
    
    def fit(self, parameters, config):
        """Train model locally on client data."""
        # Update model with server parameters
        self.model.set_weights(parameters)
        
        # Get training config from server
        local_epochs = config.get("local_epochs", 5)
        
        # Train locally
        self.model.train(self.X_train, self.y_train, epochs=local_epochs)
        
        # Compute metrics
        loss, accuracy = self.model.evaluate(self.X_train, self.y_train)
        
        # Return updated parameters and metrics
        return (
            self.model.get_weights(),  # Updated parameters
            len(self.X_train),           # Number of training examples
            {"loss": loss, "accuracy": accuracy}  # Metrics
        )
    
    def evaluate(self, parameters, config):
        """Evaluate model on client test data."""
        self.model.set_weights(parameters)
        loss, accuracy = self.model.evaluate(self.X_test, self.y_test)
        return loss, len(self.X_test), {"accuracy": accuracy}

# Create client app
def client_fn(context: Context):
    """Factory function to create client instances."""
    # Load client-specific data
    partition_id = context.node_config["partition-id"]
    X_train, y_train, X_test, y_test = load_client_data(partition_id)
    
    # Create model
    model = FraudModel()
    
    # Return client wrapped in ClientApp
    return FraudDetectionClient(model, X_train, y_train, X_test, y_test)

# Create client app
app = ClientApp(client_fn=client_fn)
'''

print("Flower Client API:")
print(flower_client_code)

## 3. Flower Server Implementation

In [ ]:
flower_server_code = '''
from flwr.server import ServerApp, ServerConfig
from flwr.server.strategy import FedAvg
from flwr.common import Context

# Define FedAvg strategy
strategy = FedAvg(
    # Fraction of clients to sample each round
    fraction_fit=0.5,
    
    # Minimum number of clients for training
    min_fit_clients=5,
    
    # Minimum number of clients for evaluation
    min_evaluate_clients=5,
    
    # Number of clients to use for evaluation
    fraction_evaluate=0.2,
    
    # Server-side learning rate (for parameter averaging)
    server_learning_rate=1.0,
    
    # Config sent to clients each round
    on_fit_config_fn=lambda round_num: {
        "local_epochs": 5,
        "learning_rate": 0.01 * (0.99 ** round_num),  # Decay
    },
)

def server_fn(context: Context):
    """Server function."""
    # Define server configuration
    config = ServerConfig(
        num_rounds=30,  # Total federated rounds
    )
    
    return ServerApp(strategy=strategy, config=config)

# Create server app
server = ServerApp(server_fn=server_fn)
'''

print("Flower Server API:")
print(flower_server_code)

## 4. Running Federated Learning

In [ ]:
run_fl_code = '''
# Option 1: Run with SuperLink (Flower's built-in server)

# Terminal 1: Start server
# flower-server-app server:app --insecure

# Terminal 2, 3, ...: Start clients
# flower-client-app client:app --insecure \\
#   --node-config partition-id=0  # For client 0
# flower-client-app client:app --insecure \\
#   --node-config partition-id=1  # For client 1

# Option 2: Run simulation (single machine)
from flwr.simulation import run_simulation

# Run FL simulation
run_simulation(
    server_app=server,
    client_app=app,
    num_supernodes=10,  # Number of clients
)
'''

print("Running Flower:")
print(run_fl_code)

## 5. Custom Aggregation Strategy

In [ ]:
custom_strategy_code = '''
from flwr.server.strategy import FedAvg
from flwr.common import Parameters, Scalar
from typing import List, Tuple, Dict, Optional

class CustomFedAvg(FedAvg):
    """Custom FedAvg with robust aggregation."""
    
    def aggregate_fit(
        self,
        server_round: int,
        results: List[Tuple[ClientProxy, FitRes]],
        failures: List[BaseException],
    ) -> Tuple[Optional[Parameters], Dict[str, Scalar]]:
        """Aggregate client updates with custom logic."""
        
        # Call parent class to get weighted average
        aggregated_parameters, metrics = super().aggregate_fit(
            server_round, results, failures
        )
        
        # Add custom logic: detect anomalous updates
        if results:
            # Extract updates from results
            updates = [fit_res.parameters for _, fit_res in results]
            
            # Detect outliers (simplified)
            # In practice, use more sophisticated detection
            n_outliers = self.detect_outliers(updates)
            
            metrics["detected_outliers"] = n_outliers
        
        return aggregated_parameters, metrics
    
    def detect_outliers(self, updates: List[Parameters]) -> int:
        """Detect anomalous client updates."""
        # Simplified outlier detection
        # In practice: use L2 norm, cosine similarity, etc.
        return 0

# Use custom strategy
strategy = CustomFedAvg(
    fraction_fit=0.5,
    min_fit_clients=5,
)
'''

print("Custom Strategy:")
print(custom_strategy_code)

## 6. Flower Features Overview

In [ ]:
# Flower feature comparison
features = {
    'Feature': [
        'Language Support',
        'Framework Support',
        'Deployment',
        'Strategies',
        'Built-in Defenses',
        'Simulation',
        'Production Ready',
        'Learning Curve'
    ],
    'Description': [
        'Python, Rust, Swift, Java, Go, JavaScript',
        'PyTorch, TensorFlow, JAX, scikit-learn',
        'SuperLink (server), client apps',
        'FedAvg, FedProx, FedAdagrad, FedYogi, custom',
        'Byzantine-robust aggregation (Krum, etc.)',
        'Single-machine simulation for research',
        'Used by major companies (e.g., banks)',
        'Easy to learn, excellent documentation'
    ]
}

df_features = pd.DataFrame(features)
print("\n" + "="*60)
print("FLOWER FRAMEWORK FEATURES")
print("="*60)
print(df_features.to_string(index=False))

## 7. Summary

### Flower Framework:

**Key Components:**
1. **Client (NumPyClient)**: Train locally, return updates
2. **Server (ServerApp)**: Coordinate training, aggregate updates
3. **Strategy**: Aggregation logic (FedAvg, FedProx, custom)
4. **Simulation**: Run FL on single machine for research

**Advantages:**
- ✅ **Framework-agnostic**: Works with any ML framework
- ✅ **Language support**: Multiple programming languages
- ✅ **Production-ready**: Robust, scalable, well-documented
- ✅ **Extensible**: Custom strategies, callbacks

**Use Cases:**
- Mobile devices (Android, iOS)
- Edge devices (IoT, embedded systems)
- Cross-silo FL (banks, hospitals)
- Research (simulation mode)

### Next Steps:
→ **Day 11**: Communication-Efficient FL (gradient compression)
→ **Day 12**: Cross-Silo Bank FL (real-world deployment)

---

**📁 Project Location**: `02_federated_learning_foundations/flower_fraud_detection/`

**📚 Documentation**: [flower.dev](https://flower.dev)